# **Reto I**

### 1. Datasets

Los datos de origen constan de dos archivos csv con la misma estructura y tipo de columnas.

* trade_details: dataset original con datos reales de operaciones financieras.
* trade_details_snapshot: copia de seguridad por posibles perdidas de datos.

### 2. Columnas y significado:

* mfamily: indica la familia de operaciones a la que pertenece.
* mgroup: indica el grupo de operaciones dentro de la familia.
* mtype: indica el tipo de operación dentro del grupo.
* origin_trade_number: indica el número de la operación de trading (la misma operación puede tener varios números de trading).
* origin_contract_number: indica el número de contrato de la operación (igual para todas las operaciones que pertenecen al mismo contrato).
* maturity: fecha de finalización del contrato de cada operación.

### 3. Descripción del problema:

En estos datasets se encuentran varias operaciones financieras de distinto tipo, que diferenciaremos mediante los distintos valores de las columnas mfamily, mgroup y mtype.

Existe un cierto tipo de operaciones especiales, llamadas FXSwaps. Estas pueden ser diferenciadas por medio de los siguientes valores:

**mfamily = CURR** \
**mgroup = FXD** \
**mtype = SWLEG**

Podemos ver en nuestro dataset que estas operaciones aparecen duplicadas, es decir, con el mismo **origin_contract_number** aunque distinto **origin_trade_number**. De estas operaciones duplicadas en origen, queremos obtener solo una de ellas.

La forma para decidir cuál de las operaciones nos interesa obtener es mediante la columna *maturity*. De ambas operaciones de trading (distinto origin_trade_number) para un mismo contrato (origin_contract_number), queremos obtener solo la *long leg*, es decir, la que tiene una mayor fecha de vencimiento (fecha más actual de la columna maturity).

Existe un cierto problema en nuestro dataset trade_details que tendremos que solucionar. Podemos ver que para algunas operaciones el campo maturity vendrá como *null*, es decir, sin informar. En estos casos, deberemos buscar esa operacion en el dataset trade_details_snapshot y el respectivo campo maturity para poder saber cuál de las dos operaciones es la *long leg* y filtrar la *short leg* 

**NOTA: Si se quiere conocer más el significado de estas operaciones financieras: https://es.wikipedia.org/wiki/Swap_(finanzas)**

### 4. Reto:

* Obtener un dataframe final donde tengamos todas las operaciones originales excepto los short leg de los contratos tipo FXSwap.
* Aunque usemos el valor de la columna maturity del dataset trade_details_snapshot en los casos que venga en la trade_details a *null*, en el dataframe final deberá venir con el valor original de trade_details.
* Hacerlo de la manera más eficiente posible a nivel computacional.

### Inicialización de SparkSession:

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

spark = (SparkSession.builder.appName("Reto 1").getOrCreate())

### Carga de CSV

In [0]:
trade_details = spark.read.option("header", "true").option("inferSchema", "true").option("delimiter", ";")\
                      .csv("dbfs:/FileStore/shared_uploads/paula.roman@bosonit.com/trade_details.csv")

trade_details_snapshot = spark.read.option("header", "true").option("inferSchema", "true").option("delimiter", ";")\
                      .csv("dbfs:/FileStore/shared_uploads/paula.roman@bosonit.com/trade_details_snapshot.csv")

trade_details.show(50)
trade_details_snapshot.show(50)

+-------+------+-----+-------------------+----------------------+----------+
|mfamily|mgroup|mtype|origin_trade_number|origin_contract_number|  maturity|
+-------+------+-----+-------------------+----------------------+----------+
|    IRD|  BOND| null|          316391872|             678876251|2021-09-22|
|   CURR|   FXD|  FXD|           32734782|              54853428|2021-09-22|
|    IRD| LN_BR| null|               1111|                  2222|2022-10-06|
|    IRD|   IRS| null|            2222222|               2222222|2024-10-15|
|    SCF|   SCF|  SCF|            3815982|               3672136|      NULL|
|    IRD| LN_BR| null|           14596583|              13774383|2020-12-29|
|    IRD|   IRS| null|           18343978|              17356077|2024-10-15|
|    IRD| LN_BR| null|           19203839|              18176215|2022-10-06|
|    IRD|    CF| null|           20513130|              19433281|2021-07-06|
|    IRD|    CF| null|           20533916|              19453781|2023-07-06|

### Resultado:

**INSTRUCCIONES**: El DataFrame resultante debe almacenarse en la variable `resultado`, sustituyendo el valor `None` por el código que consideréis oportuno. De esta forma podréis comprobar si el resultado es correcto.

In [0]:
joined_df = trade_details.join(trade_details_snapshot.select(trade_details_snapshot["mfamily"].alias("mfam"), trade_details_snapshot["mgroup"].alias("mgr"),\
                                                             trade_details_snapshot["mtype"].alias("mty"), trade_details_snapshot["origin_contract_number"].alias("cont_num"),                                                                  trade_details_snapshot["origin_trade_number"], trade_details_snapshot["maturity"].alias("mat")),\
                               trade_details["origin_trade_number"] == trade_details_snapshot["origin_trade_number"])

clean_df = joined_df.drop(joined_df["mfam"]).drop(joined_df["mgr"]).drop(joined_df["mty"]).drop(trade_details_snapshot["origin_trade_number"])\
                    .drop(joined_df["cont_num"])

clean_df.show()

+-------+------+-----+-------------------+----------------------+----------+-------------------+
|mfamily|mgroup|mtype|origin_trade_number|origin_contract_number|  maturity|                mat|
+-------+------+-----+-------------------+----------------------+----------+-------------------+
|    IRD|  BOND| null|          316391872|             678876251|2021-09-22|2021-09-22 00:00:00|
|   CURR|   FXD|  FXD|           32734782|              54853428|2021-09-22|2021-09-22 00:00:00|
|    IRD| LN_BR| null|               1111|                  2222|2022-10-06|2022-10-06 00:00:00|
|    IRD|   IRS| null|            2222222|               2222222|2024-10-15|2024-10-15 00:00:00|
|    SCF|   SCF|  SCF|            3815982|               3672136|      NULL|2021-05-04 00:00:00|
|    IRD| LN_BR| null|           14596583|              13774383|2020-12-29|2020-12-29 00:00:00|
|    IRD|   IRS| null|           18343978|              17356077|2024-10-15|2024-10-15 00:00:00|
|    IRD| LN_BR| null|        

In [0]:
new_trade_df = clean_df.withColumn("new_maturity", F.when(F.col("maturity") == "NULL", F.col("mat"))\
                                                    .otherwise(F.col("maturity")))\
                       .withColumn("new_maturity", F.date_format("new_maturity","yyyy-MM-dd"))\
                       .drop(clean_df["mat"])

new_trade_df.show()

+-------+------+-----+-------------------+----------------------+----------+------------+
|mfamily|mgroup|mtype|origin_trade_number|origin_contract_number|  maturity|new_maturity|
+-------+------+-----+-------------------+----------------------+----------+------------+
|    IRD|  BOND| null|          316391872|             678876251|2021-09-22|  2021-09-22|
|   CURR|   FXD|  FXD|           32734782|              54853428|2021-09-22|  2021-09-22|
|    IRD| LN_BR| null|               1111|                  2222|2022-10-06|  2022-10-06|
|    IRD|   IRS| null|            2222222|               2222222|2024-10-15|  2024-10-15|
|    SCF|   SCF|  SCF|            3815982|               3672136|      NULL|  2021-05-04|
|    IRD| LN_BR| null|           14596583|              13774383|2020-12-29|  2020-12-29|
|    IRD|   IRS| null|           18343978|              17356077|2024-10-15|  2024-10-15|
|    IRD| LN_BR| null|           19203839|              18176215|2022-10-06|  2022-10-06|
|    IRD| 

In [0]:
from pyspark.sql import Window
windowContratos = Window().partitionBy("origin_contract_number")

new_trade_max_df = new_trade_df.withColumn("mat_max", F.when((F.col("mfamily") == "CURR") & (F.col("mgroup") == "FXD") & (F.col("mtype") == "SWLEG"),\
                                                      F.max(F.col("new_maturity")).over(windowContratos)))

new_trade_max_df.show(100)

+-------+------+-----+-------------------+----------------------+----------+------------+----------+
|mfamily|mgroup|mtype|origin_trade_number|origin_contract_number|  maturity|new_maturity|   mat_max|
+-------+------+-----+-------------------+----------------------+----------+------------+----------+
|    IRD| LN_BR| null|               1111|                  2222|2022-10-06|  2022-10-06|      null|
|    IRD|   IRS| null|            2222222|               2222222|2024-10-15|  2024-10-15|      null|
|    SCF|   SCF|  SCF|            3815982|               3672136|      NULL|  2021-05-04|      null|
|    EQD| EQUIT|  FWD|           10000001|              10000001|2019-07-02|  2019-07-02|      null|
|    IRD|  BOND|  FWD|           10000009|              10000009|2021-06-12|  2021-06-12|      null|
|    IRD| LN_BR| null|           14596583|              13774383|2020-12-29|  2020-12-29|      null|
|    IRD|   IRS| null|           18343978|              17356077|2024-10-15|  2024-10-15|  

In [0]:
trade_df = new_trade_max_df.filter((F.col("mat_max").isNull()) | (F.col("new_maturity") == F.col("mat_max")))

resultado = trade_df.drop(trade_df["mat_max"]).drop(trade_df["new_maturity"])

resultado.show()

+-------+------+-----+-------------------+----------------------+----------+
|mfamily|mgroup|mtype|origin_trade_number|origin_contract_number|  maturity|
+-------+------+-----+-------------------+----------------------+----------+
|    IRD| LN_BR| null|               1111|                  2222|2022-10-06|
|    IRD|   IRS| null|            2222222|               2222222|2024-10-15|
|    SCF|   SCF|  SCF|            3815982|               3672136|      NULL|
|    EQD| EQUIT|  FWD|           10000001|              10000001|2019-07-02|
|    IRD|  BOND|  FWD|           10000009|              10000009|2021-06-12|
|    IRD| LN_BR| null|           14596583|              13774383|2020-12-29|
|    IRD|   IRS| null|           18343978|              17356077|2024-10-15|
|    IRD| LN_BR| null|           19203839|              18176215|2022-10-06|
|   CURR|   FXD|SWLEG|           19665186|              18622136|2020-12-30|
|   CURR|   FXD|SWLEG|           19772400|              18724280|2021-11-05|

Ejecuta la siguiente celda (no modifiques su código) y te dirá si tu solución es correcta o no. En caso de ser correcta, se ejecutará correctamente y no mostrará nada, pero si no lo es mostrará un error. Además de esas pruebas, se realizarán algunas más (ocultas) a la hora de puntuar el ejercicio, pero evaluar dicha celda es un indicador bastante fiable acerca de si realmente has implementado la solución correcta o no.

In [0]:
assert(resultado.count() == 26)
assert(resultado.orderBy("origin_contract_number").collect()[24][4] == 564367838)
assert(resultado.orderBy("origin_contract_number").collect()[19][5] == "NULL")
assert(resultado.orderBy("origin_trade_number").collect()[16][5] == "NULL")

---------------------------------------------------------------------------
AssertionError                            Traceback (most recent call last)
<command-2891414116371057> in <module>
      1 assert(resultado.count() == 26)
      2 assert(resultado.orderBy("origin_contract_number").collect()[24][4] == 564367838)
----> 3 assert(resultado.orderBy("origin_contract_number").collect()[19][5] == "NULL")
      4 assert(resultado.orderBy("origin_trade_number").collect()[16][5] == "NULL")

AssertionError: 